## PLOTS DE PRUEBA SOBRE EL DATASET CALIFORNIA

In [2]:
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [3]:
import pandas as pd
import math
import numpy as np
import math


def lat_to_meters(latitude):
    origin_shift = 2 * math.pi * 6378137 / 2.0
    y = np.vectorize(lambda lat: math.log(math.tan((90 + lat) * math.pi / 360.0)) / (math.pi / 180.0))(latitude)
    y = y * origin_shift / 180.0
    return y


def lon_to_meters(longitude):
    origin_shift = 2 * math.pi * 6378137 / 2.0
    x = longitude * origin_shift / 180.0
    return x


data = pd.read_csv('./data/california_cities.csv')
print(data)

def lon_lat_to_meters(longitude, latitude):
    origin_shift = 2 * math.pi * 6378137 / 2.0
    x = longitude * origin_shift / 180.0
    y = math.log(math.tan((90 + latitude) * math.pi / 360.0)) / (math.pi / 180.0)
    y = y * origin_shift / 180.0
    return x, y


data['longd'] = lon_to_meters(np.array(data['longd']))
data['latd'] = lat_to_meters(np.array(data['latd']))

     Unnamed: 0         city       latd       longd  elevation_m   
0             0     Adelanto  34.576111 -117.432778        875.0  \
1             1  AgouraHills  34.153333 -118.761667        281.0   
2             2      Alameda  37.756111 -122.274444          NaN   
3             3       Albany  37.886944 -122.297778          NaN   
4             4     Alhambra  34.081944 -118.135000        150.0   
..          ...          ...        ...         ...          ...   
477         477   Yountville  38.403056 -122.362222         30.0   
478         478        Yreka  41.726667 -122.637500        787.0   
479         479     YubaCity  39.134722 -121.626111         18.0   
480         480      Yucaipa  34.030278 -117.048611        798.0   
481         481  YuccaValley  34.133333 -116.416667       1027.0   

     elevation_ft  population_total  area_total_sq_mi  area_land_sq_mi   
0          2871.0             31765            56.027           56.009  \
1           922.0             20330

In [4]:
min_long = min(data.longd)-20000
min_lat = min(data.latd)-20000
max_long = max(data.longd)+20000
max_lat = max(data.latd)+20000
print(min_long,max_long,min_lat,max_lat)

-13852930.990941485 -12736811.657859063 3819467.585782222 5175738.054781796


In [16]:
from bokeh.plotting import figure
from bokeh.models import WMTSTileSource
import numpy as np
from bokeh.models.mappers import ColorMapper, LinearColorMapper
from bokeh.palettes import Viridis5

# Coordenadas de California en grados decimales
california_bounds= x_range,y_range = ((min_long,max_long), (min_lat,max_lat))



p = figure(tools='pan, wheel_zoom', x_range=x_range, y_range=y_range, 
           x_axis_type="mercator", y_axis_type="mercator")

url = 'http://a.basemaps.cartocdn.com/rastertiles/voyager/{Z}/{X}/{Y}.png'
attribution = "Tiles by Carto, under CC BY 3.0. Data by OSM, under ODbL"

p.add_tile(WMTSTileSource(url=url, attribution=attribution))
pobl_norm =  data.population_total / np.max(data.population_total)
data.elevation_m = np.nan_to_num(data.elevation_m, nan=0)
elev_norm =  data.elevation_m / np.max(data.elevation_m)
color_mapper = LinearColorMapper(palette=Viridis5)

# Agregar un conjunto de coordenadas a la gráfica
p.circle(x=data.longd, y=data.latd, size=300*pobl_norm, color='red',alpha= 1/(10* pobl_norm))
#size=size, color='red', alpha=alpha
show(p)




Hemos graficado usando la transparencia y el tamaño la población de cada ciudad. Hemos utizado ambas ya que así cuanto mayor sea el círculo mayor es su transparencia y así se pueden superponer y podemos ver con cierta claridad.

----------------------

In [58]:
from bokeh.plotting import figure
from bokeh.models import WMTSTileSource
import numpy as np
from bokeh.palettes import Category10
from bokeh.transform import factor_cmap, linear_cmap
from bokeh.palettes import Blues8
from bokeh.models import LogColorMapper,ColorBar


# Coordenadas de California en grados decimales
california_bounds= x_range,y_range = ((min_long,max_long), (min_lat,max_lat))



p = figure(tools='pan, wheel_zoom', x_range=x_range, y_range=y_range, 
           x_axis_type="mercator", y_axis_type="mercator")

url = 'http://a.basemaps.cartocdn.com/rastertiles/voyager/{Z}/{X}/{Y}.png'
attribution = "Tiles by Carto, under CC BY 3.0. Data by OSM, under ODbL"

p.add_tile(WMTSTileSource(url=url, attribution=attribution))
area =  data.area_total_km2 / np.max(data.area_total_km2)
water =  data.area_water_percent / np.max(data.area_water_percent)
data['water']=water


# Agregar un conjunto de coordenadas a la gráfica
p.circle(x=data.longd, y=data.latd, size=300*area,alpha= 1/(1000* water))

show(p)


En este plot procedemos de forma muy similar, aunque ahora el tamaño es la cantidad de area total y la transparencia nos indica su porcentaje de agua.

----

Finalmente realizamos el mismo gráfico anterior pero con la transparencia y el color.

In [59]:


california_bounds= x_range,y_range = ((min_long,max_long), (min_lat,max_lat))



p = figure(tools='pan, wheel_zoom', x_range=x_range, y_range=y_range, 
           x_axis_type="mercator", y_axis_type="mercator")

url = 'http://a.basemaps.cartocdn.com/rastertiles/voyager/{Z}/{X}/{Y}.png'
attribution = "Tiles by Carto, under CC BY 3.0. Data by OSM, under ODbL"

p.add_tile(WMTSTileSource(url=url, attribution=attribution))
pobl_norm =  data.population_total / np.max(data.population_total)
data.elevation_m = np.nan_to_num(data.elevation_m, nan=0)
elev_norm =  data.elevation_m / np.max(data.elevation_m)
color_mapper = LogColorMapper(palette=Viridis5)

data_dict=dict(
    x=data.longd,
    y=data.latd,
    land = 300*data.area_land_km2,
    water = 1/(1000*data.area_water_percent)
)


p.circle(source = data_dict, x='x', y='y', fill_alpha= 'water', fill_color = {'field': 'land', 'transform': color_mapper} ,size = 20)
cb = ColorBar(color_mapper = color_mapper, location = (5,6))
 
p.add_layout(cb, 'right')
show(p)

